<a href="https://colab.research.google.com/github/Gabrielnkl/notebooks/blob/main/TensorFlow_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification with TensorFlow: From Classical ML to Deep Learning

This notebook covers:
- Baseline Model (Without Deep Learning)
- CNN Model from Scratch
- Transfer Learning
- Fine-tuning Pre-trained Model

In [ ]:
!pip install tensorflow scikit-learn tensorflow-datasets

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import tensorflow_datasets as tfds

## Load and preprocess CIFAR-10 dataset

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    as_supervised=True,
    with_info=True,
)

def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

BATCH_SIZE = 64

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.JHVKC8_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.JHVKC8_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


## 1. Baseline Model (Without Deep Learning)

In [3]:
X_train, y_train = [], []
for img, label in tfds.as_numpy(ds_train):
    X_train.append(img.reshape(-1))
    y_train.append(label)

X_test, y_test = [], []
for img, label in tfds.as_numpy(ds_test):
    X_test.append(img.reshape(-1))
    y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

baseline_model = LogisticRegression(max_iter=100)
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (782,) + inhomogeneous part.

In [4]:
X_train, y_train = [], []
for img_batch, label_batch in tfds.as_numpy(ds_train):
    # Iterate over individual images and labels within the batch
    for img, label in zip(img_batch, label_batch):
        X_train.append(img.reshape(-1))
        y_train.append(label)

X_test, y_test = [], []
for img_batch, label_batch in tfds.as_numpy(ds_test):
    # Iterate over individual images and labels within the batch
    for img, label in zip(img_batch, label_batch):
        X_test.append(img.reshape(-1))
        y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

baseline_model = LogisticRegression(max_iter=100)
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Baseline Accuracy: 0.4051


## 2. CNN Model from Scratch

In [5]:
cnn_model = Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(ds_train, epochs=10, validation_data=ds_test)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.3826 - loss: 1.6943 - val_accuracy: 0.5771 - val_loss: 1.1938
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5949 - loss: 1.1578 - val_accuracy: 0.6318 - val_loss: 1.0556
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6535 - loss: 0.9955 - val_accuracy: 0.6675 - val_loss: 0.9652
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6926 - loss: 0.8945 - val_accuracy: 0.6736 - val_loss: 0.9352
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7207 - loss: 0.8129 - val_accuracy: 0.6852 - val_loss: 0.9137
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7420 - loss: 0.7404 - val_accuracy: 0.6903 - val_loss: 0.9097
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7656 - loss: 0.6685 - val_accuracy: 0.6942 - val_loss: 0.9238
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7902 - loss: 0.6059 - val_accuracy: 0

## 3. Transfer Learning (Feature Extraction)

In [6]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

def preprocess_mobilenet(image, label):
    image = tf.image.resize(image, (96, 96))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train_mobilenet = ds_train.map(preprocess_mobilenet)
ds_test_mobilenet = ds_test.map(preprocess_mobilenet)

model_transfer = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_transfer.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

model_transfer.fit(ds_train_mobilenet, epochs=5, validation_data=ds_test_mobilenet)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.1004 - loss: 2.3401 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 20ms/step - accuracy: 0.0974 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.0983 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.0984 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.0982 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026


## 4. Fine-Tuning the Pre-trained Model

In [7]:
base_model.trainable = True

model_transfer.compile(optimizer=keras.optimizers.Adam(1e-5),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

model_transfer.fit(ds_train_mobilenet, epochs=5, validation_data=ds_test_mobilenet)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 115s 88ms/step - accuracy: 0.3409 - loss: 1.8949 - val_accuracy: 0.1000 - val_loss: 2.5275
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 94s 53ms/step - accuracy: 0.7351 - loss: 0.7870 - val_accuracy: 0.0995 - val_loss: 2.5636
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 54ms/step - accuracy: 0.8132 - loss: 0.5492 - val_accuracy: 0.1008 - val_loss: 2.5581
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.8545 - loss: 0.4283 - val_accuracy: 0.0999 - val_loss: 2.5333
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.8854 - loss: 0.3426 - val_accuracy: 0.1014 - val_loss: 2.8769


## Results Comparison

In [8]:
cnn_loss, cnn_acc = cnn_model.evaluate(ds_test)
transfer_loss, transfer_acc = model_transfer.evaluate(ds_test_mobilenet)

print(f"Baseline Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"CNN Accuracy: {cnn_acc:.2f}")
print(f"Transfer Learning Accuracy: {transfer_acc:.2f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6987 - loss: 0.9820
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.1033 - loss: 2.8581
Baseline Accuracy: 0.41
CNN Accuracy: 0.70
Transfer Learning Accuracy: 0.10
